In [20]:
import SimpleITK as sitk
import numpy as np

def fill_and_balance_volume(mask_img, dilate_radius=3, max_iter=10):
    mask = sitk.Cast(mask_img > 0, sitk.sitkUInt8)
    original_vol = np.count_nonzero(sitk.GetArrayFromImage(mask))

    dilated = sitk.BinaryDilate(mask, [dilate_radius]*3)

    eroded = dilated
    prev_diff = float('inf')
    best = eroded

    for r in range(1, max_iter+1):
        test = sitk.BinaryErode(dilated, [r]*3)
        vol = np.count_nonzero(sitk.GetArrayFromImage(test))
        diff = abs(vol - original_vol)
        if diff < prev_diff:
            best, prev_diff = test, diff
        else:
            break

    return best

mandible = sitk.ReadImage(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_totalseg.nii.gz")
filled_balanced = fill_and_balance_volume(mandible, dilate_radius=5)
sitk.WriteImage(filled_balanced, r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_filled_balanced_r3.nii.gz")
print("Done")


Done


In [23]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\CT.nii.gz")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Normalize to [0, 1] (binary)

seg_paths = {
    "mandible_manual": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_manual.nii.gz",
    #"mandible_totalseg": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_totalseg.nii.gz",
    #"mandible_totalseg_dilate_contract": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\totalseg_mandible_filled_dilate_contract.nii.gz",
    #"mandible_totalseg_dilate_contract": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\totalseg_mandible_filled_dilate_contract.nii.gz",
    "mandible_filled_balanced": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_filled_balanced_r3.nii.gz",
}

segments = {name: nib.load(path).get_fdata() for name, path in seg_paths.items()}
spacing = ct_img.header.get_zooms()

overlay_colors = {
    "mandible_manual": (0.2, 1.0, 0.2, 0.35),          # green
    #"mandible_totalseg": (1.0, 0.2, 0.2, 0.25),       # reddish
    #"mandible_totalseg_dilate_contract": (0.2, 1.0, 0.2, 0.35), # green
    "mandible_filled_balanced": (0.2, 0.4, 1.0, 0.25), # blue
}

def show_slice(slice_index, plane, alpha=0.35, patient_id="UIDQQ0x7axQ0Q1"):
    plt.figure(figsize=(6, 6))

    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
        seg_slices = {n: np.rot90(S[:, :, slice_index]) for n, S in segments.items()}

    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
        seg_slices = {n: np.flipud(np.rot90(S[:, slice_index, :])) for n, S in segments.items()}

    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
        seg_slices = {n: np.flipud(np.rot90(S[slice_index, :, :])) for n, S in segments.items()}

    plt.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name, seg_slice in seg_slices.items():
        mask = seg_slice > 0.5
        if np.any(mask):
            rgba = list(overlay_colors[name])
            rgba[3] = alpha  
            overlay = np.zeros((*mask.shape, 4))
            overlay[mask] = rgba
            plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")


    legend_handles = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in overlay_colors
    ]
    leg=plt.legend(handles=legend_handles, loc='upper right', fontsize=8, frameon=False, labelcolor='white', title="Segments")
    leg.get_title().set_fontsize(10)
    leg.get_title().set_color('white')
    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2


plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown, alpha=alpha_slider)


interactive(children=(IntSlider(value=120, description='Slice:', layout=Layout(width='80%'), max=239), Dropdow…

<function __main__.show_slice(slice_index, plane, alpha=0.35, patient_id='UIDQQ0x7axQ0Q1')>

In [19]:
#to view on subplots
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\CT.nii.gz")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Normalize to [0, 1]
spacing = ct_img.header.get_zooms()

seg_paths = {
    "mandible_manual": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_manual.nii.gz",
    #"mandible_totalseg": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_totalseg.nii.gz",
    #"mandible_totalseg_dilate_contract": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\totalseg_mandible_filled_dilate_contract.nii.gz"
    "mandible_filled_balanced": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_filled_balanced_r3.nii.gz",
    #"teeth": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\teeth.nii.gz",
}
segments = {name: nib.load(path).get_fdata() for name, path in seg_paths.items()}

overlay_colors = {
    "mandible_manual": (0.2, 0.4, 1.0, 0.25),   # blue
    "mandible_filled_balanced": (0.2, 0.4, 1.0, 0.25),
   # "Dentition": (1.0, 0.6, 0.0, 0.35),         # orange
    #"teeth": (1.0, 0.6, 0.0, 0.35),
}

def show_slice(slice_index, plane, alpha=0.35):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    plt.tight_layout()

    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
        seg_slices = {n: np.rot90(S[:, :, slice_index]) for n, S in segments.items()}

    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
        seg_slices = {n: np.flipud(np.rot90(S[:, slice_index, :])) for n, S in segments.items()}

    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
        seg_slices = {n: np.flipud(np.rot90(S[slice_index, :, :])) for n, S in segments.items()}

    # left subplot 
    ax_left = axes[0]
    ax_left.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name in ["mandible_manual"]:
        if name in seg_slices:
            mask = seg_slices[name] > 0.5
            if np.any(mask):
                rgba = list(overlay_colors[name])
                rgba[3] = alpha
                overlay = np.zeros((*mask.shape, 4))
                overlay[mask] = rgba
                ax_left.imshow(overlay, origin='lower', aspect=aspect)

    ax_left.set_title(f"{plane} slice {slice_index} — Manual Segmentation")
    ax_left.axis("off")

    legend_handles_left = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in ["mandible_manual"]
    ]
    leg_left = ax_left.legend(
        handles=legend_handles_left,
        loc='lower right',
        fontsize=8,
        frameon=True,
        facecolor='black',
        edgecolor='white',
        labelcolor='white',
        title="Segments",
        title_fontsize=9
    )

    plt.setp(leg_left.get_texts(), color='white')
    leg_left.get_title().set_color('white')

    # right subplot
    ax_right = axes[1]
    ax_right.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name in ["mandible_filled_balanced"]:
        if name in seg_slices:
            mask = seg_slices[name] > 0.5
            if np.any(mask):
                rgba = list(overlay_colors[name])
                rgba[3] = alpha
                overlay = np.zeros((*mask.shape, 4))
                overlay[mask] = rgba
                ax_right.imshow(overlay, origin='lower', aspect=aspect)

    ax_right.set_title(f"{plane} slice {slice_index} — Filled TotalSeg Segmentation")
    ax_right.axis("off")

    legend_handles_right = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in ["mandible_filled_balanced"]
    ]
    leg_right = ax_right.legend(
        handles=legend_handles_right,
        loc='lower right',
        fontsize=8,
        frameon=True,
        facecolor='black',
        edgecolor='white',
        labelcolor='white',
        title="Segments",
        title_fontsize=9
    )

    plt.setp(leg_right.get_texts(), color='white')
    leg_right.get_title().set_color('white')

    plt.subplots_adjust(wspace=0.02)
    clear_output(wait=True)
    display(fig)
    plt.close(fig)


plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)

def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2

plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown, alpha=alpha_slider)


interactive(children=(IntSlider(value=120, description='Slice:', layout=Layout(width='80%'), max=239), Dropdow…

<function __main__.show_slice(slice_index, plane, alpha=0.35)>